In [40]:
import os, nltk, collections
import pandas as pd
import collections
from nltk.tokenize import word_tokenize, sent_tokenize
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from ast import literal_eval
data = pd.read_csv("small_datasetprocessed.csv")
X = data["content_stemmed"].apply(lambda x:literal_eval(x))#ikke sikker på de skal være i en liste
#tjek lige om der er andre mærkelige kolonner vi har lavet i det?
y = data["type"] #eller type?

X_train, X_val, y_train, y_val= train_test_split(X,y,test_size=0.2,random_state=0,shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val,y_val, test_size=0.5,random_state=0,shuffle=True) 


In [41]:
print(type(X[0]))


<class 'list'>


In [55]:
tfIdfTransformer = TfidfTransformer(use_idf=True)
countVectorizer = CountVectorizer()
X_BoW = countVectorizer.fit_transform(X_train.toarray())

#det virker lidt som om den antager at den ikke får en tokenized tekst?

AttributeError: 'Series' object has no attribute 'toarray'

In [63]:
def fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fun,
    preprocessor=fun,
    token_pattern=None)  

tfidf.fit(X_train) #vær opmærksom på at vocab nu er relativt til træningssættet, men det skal det jo sådan set også være, ellers ville der være leakage
tfidf.vocabulary_
X_tfidf = X_train.apply(lambda x: tfidf.transform([x]))
#X_tfidf = X_train["content"].apply(lambda x: tfidf.transform([x]))
#måske med apply?

In [65]:
BoW = CountVectorizer(
    analyzer='word',
    tokenizer=fun,
    preprocessor=fun,
    token_pattern=None)  

BoW.fit(X_train) #vær opmærksom på at vocab nu er relativt til træningssættet, men det skal det jo sådan set også være, ellers ville der være leakage
BoW.vocabulary_
X_BoW = X_train.apply(lambda x: BoW.transform([x]))


print(BoW.transform([X_train[0]]))

#tror det er rigtigt??


  (0, 36)	11
  (0, 114)	5
  (0, 115)	2
  (0, 213)	1
  (0, 283)	1
  (0, 345)	1
  (0, 650)	1
  (0, 762)	1
  (0, 861)	1
  (0, 884)	1
  (0, 1013)	1
  (0, 1088)	1
  (0, 1134)	1
  (0, 1323)	1
  (0, 1431)	2
  (0, 1446)	4
  (0, 1453)	3
  (0, 1662)	2
  (0, 1668)	1
  (0, 1739)	6
  (0, 2002)	1
  (0, 2049)	1
  (0, 2082)	4
  (0, 2120)	1
  (0, 2134)	1
  :	:
  (0, 7999)	1
  (0, 8034)	1
  (0, 8038)	1
  (0, 8077)	2
  (0, 8116)	1
  (0, 8321)	1
  (0, 8330)	1
  (0, 8481)	1
  (0, 8614)	1
  (0, 8673)	3
  (0, 8735)	1
  (0, 8832)	1
  (0, 8990)	1
  (0, 8993)	2
  (0, 9054)	1
  (0, 9302)	1
  (0, 9323)	1
  (0, 9343)	4
  (0, 9363)	1
  (0, 9515)	1
  (0, 9577)	1
  (0, 9584)	1
  (0, 9586)	4
  (0, 9602)	1
  (0, 9651)	1
